### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish Spanish"

# parameter
file = "Turkish Spanish SV"
sheets = "2 gram hybrid"  # 2 gram target, 2 gram hybrid
start_first_column = 0  # start column location number
time_shift = 0.6
sample_num = 6

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [7]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [8]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [9]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [10]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select = strip(df_word_group_select)
df_word_group_select

,twogram_pair1
0,ajan rolü
1,alarm şifresi
2,alerjik şok
3,arkeoloji profesörü
4,avukatlar ofisi
...,...
247,trafik lambası
248,trafik polisi
249,tren istasyonu
250,turbo motoru


In [11]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[start_first_column:]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [12]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,biletleri kontrol,1070.149,1154.215,gelip biletleri kontrol etti arkadaş demin kon...,HKfLNSElw0w
1,biletleri kontrol,669.840,683.080,görevli abimiz gelip biletleri kontrol ettiyor...,e18XkZwVzTE
2,ekip lideri,214.630,221.200,rahatsa bir işveren olarak veya bir ekip lider...,fGbeuBzH6_4
3,elektrik motorları,199.395,202.405,arka tekerler üzerinde bulunan elektrik motorl...,RIfMsffY7dU
4,elektrik motorları,314.815,317.825,başladıktan sonra elektrik motorları ile birlikte,RIfMsffY7dU
...,...,...,...,...,...
328,tüp bebek,16.480,24.520,şu andaki konuğumuz operatör doktor tolga ecem...,pQpngpFZjGk
329,tüp bebek,113.800,120.340,tüp bebek yaptırır gebe kalırım anlayışına sah...,0Apxzi5__Hw
330,tüp bebek,244.040,251.060,tüp bebek sürecimizi başladı işte türkiyede bi...,GWzi144zrEs
331,tüp bebek,2849.787,2852.237,tüp bebek için çektik biz o paraları,GJyk2ANh7mg


In [13]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biletleri kontrol,1073.651750,1086.962200,gelip biletleri kontrol etti arkadaş demin kon...,HKfLNSElw0w
1,biletleri kontrol,672.436078,674.902353,görevli abimiz gelip biletleri kontrol ettiyor...,e18XkZwVzTE
2,ekip lideri,217.303837,218.296977,rahatsa bir işveren olarak veya bir ekip lider...,fGbeuBzH6_4
3,elektrik motorları,200.951897,201.989828,arka tekerler üzerinde bulunan elektrik motorl...,RIfMsffY7dU
4,elektrik motorları,315.859286,317.087857,başladıktan sonra elektrik motorları ile birlikte,RIfMsffY7dU
...,...,...,...,...,...
328,tüp bebek,22.827368,23.758316,şu andaki konuğumuz operatör doktor tolga ecem...,pQpngpFZjGk
329,tüp bebek,113.800000,114.428846,tüp bebek yaptırır gebe kalırım anlayışına sah...,0Apxzi5__Hw
330,tüp bebek,244.040000,244.778947,tüp bebek sürecimizi başladı işte türkiyede bi...,GWzi144zrEs
331,tüp bebek,2849.787000,2850.467556,tüp bebek için çektik biz o paraları,GJyk2ANh7mg


In [14]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biletleri kontrol,1073.051750,1087.562200,gelip biletleri kontrol etti arkadaş demin kon...,HKfLNSElw0w
1,biletleri kontrol,671.836078,675.502353,görevli abimiz gelip biletleri kontrol ettiyor...,e18XkZwVzTE
2,ekip lideri,216.703837,218.896977,rahatsa bir işveren olarak veya bir ekip lider...,fGbeuBzH6_4
3,elektrik motorları,200.351897,202.589828,arka tekerler üzerinde bulunan elektrik motorl...,RIfMsffY7dU
4,elektrik motorları,315.259286,317.687857,başladıktan sonra elektrik motorları ile birlikte,RIfMsffY7dU
...,...,...,...,...,...
328,tüp bebek,22.227368,24.358316,şu andaki konuğumuz operatör doktor tolga ecem...,pQpngpFZjGk
329,tüp bebek,113.200000,115.028846,tüp bebek yaptırır gebe kalırım anlayışına sah...,0Apxzi5__Hw
330,tüp bebek,243.440000,245.378947,tüp bebek sürecimizi başladı işte türkiyede bi...,GWzi144zrEs
331,tüp bebek,2849.187000,2851.067556,tüp bebek için çektik biz o paraları,GJyk2ANh7mg


In [15]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,biletleri kontrol,1073,1088,gelip biletleri kontrol etti arkadaş demin kon...,HKfLNSElw0w
1,biletleri kontrol,672,676,görevli abimiz gelip biletleri kontrol ettiyor...,e18XkZwVzTE
2,ekip lideri,217,219,rahatsa bir işveren olarak veya bir ekip lider...,fGbeuBzH6_4
3,elektrik motorları,200,203,arka tekerler üzerinde bulunan elektrik motorl...,RIfMsffY7dU
4,elektrik motorları,315,318,başladıktan sonra elektrik motorları ile birlikte,RIfMsffY7dU
...,...,...,...,...,...
328,tüp bebek,22,24,şu andaki konuğumuz operatör doktor tolga ecem...,pQpngpFZjGk
329,tüp bebek,113,115,tüp bebek yaptırır gebe kalırım anlayışına sah...,0Apxzi5__Hw
330,tüp bebek,243,245,tüp bebek sürecimizi başladı işte türkiyede bi...,GWzi144zrEs
331,tüp bebek,2849,2851,tüp bebek için çektik biz o paraları,GJyk2ANh7mg


In [16]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,biletleri kontrol,1073,1088,gelip biletleri kontrol etti arkadaş demin kon...,HKfLNSElw0w,https://www.youtube.com/watch?v=HKfLNSElw0w&t=...
1,biletleri kontrol,672,676,görevli abimiz gelip biletleri kontrol ettiyor...,e18XkZwVzTE,https://www.youtube.com/watch?v=e18XkZwVzTE&t=...
2,ekip lideri,217,219,rahatsa bir işveren olarak veya bir ekip lider...,fGbeuBzH6_4,https://www.youtube.com/watch?v=fGbeuBzH6_4&t=...
3,elektrik motorları,200,203,arka tekerler üzerinde bulunan elektrik motorl...,RIfMsffY7dU,https://www.youtube.com/watch?v=RIfMsffY7dU&t=...
4,elektrik motorları,315,318,başladıktan sonra elektrik motorları ile birlikte,RIfMsffY7dU,https://www.youtube.com/watch?v=RIfMsffY7dU&t=...
...,...,...,...,...,...,...
328,tüp bebek,22,24,şu andaki konuğumuz operatör doktor tolga ecem...,pQpngpFZjGk,https://www.youtube.com/watch?v=pQpngpFZjGk&t=22s
329,tüp bebek,113,115,tüp bebek yaptırır gebe kalırım anlayışına sah...,0Apxzi5__Hw,https://www.youtube.com/watch?v=0Apxzi5__Hw&t=...
330,tüp bebek,243,245,tüp bebek sürecimizi başladı işte türkiyede bi...,GWzi144zrEs,https://www.youtube.com/watch?v=GWzi144zrEs&t=...
331,tüp bebek,2849,2851,tüp bebek için çektik biz o paraları,GJyk2ANh7mg,https://www.youtube.com/watch?v=GJyk2ANh7mg&t=...


In [17]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{sheets.capitalize()}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [18]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Spanish_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [19]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [20]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass